<h1>Snow Clearance Fines, 2019-2023</h1>
15 February 2024

This analysis looks at fines levied for uncleared sidewalks, based on FOIA data requested from the Department of Administrative Hearings (H064920-011124.xlsx). This dataset contained 3058 records dating from 1/1/2001 to 9/12/2023; filtered for those between 7/1/2019 and 6/30/2023 has 2560 records. Four of these could not be geocoded due to "unknown" address.<br>

### Record Count
<ul>
    <li>2556 valid fines records from July 1 2019 to June 30 2023
        <li>1912 dockets. Some dockets contain multiple fines records, which are identical except for a different fine amount in each record.
            <li>1735 addresses. some addresses have been fined by both CDOT and Streets & Sanitation, with a separate court docket for each.
                <li>some addresses have been fined by multiple agencies
    </ul>

### Preliminary Findings    
<ul>
    <li>73% of court dockets were issued by CDOT, 26% by Streets and Sanitation. The remaining 1% were issued by the police or Business Affairs and Consumer Protection
    <li>Englewood, Garfield Ridge, and West Englewood are the three communities with the highest number of dockets per capita
    <li>Only 25 court dockets were issued by police. West Englewood, Englewood, and Garfield Ridge have the highest rates and account for half the dockets citywide.
        <li>For dockets issued by CDOT, Garfield Ridge, Grand Boulevard, and Armour Square have the highest per capita rate
            <li>For dockets issued by Streets and Sanitation, Englewood, West Englewood, and Brighton Park have the highest per capita rate
    </ul>

<a name="read"></a>
# 1. Read and Prepare Geocoded Fines Data

In [1]:
import pandas as pd
import altair as alt
import numpy as np

Note the following data preparation steps prior to this notebook
<ol>
<li>Prepared data by parsing dates and correcting data entry errors in addresses; see <a href="fines-01-prep-data.ipynb">fines-01-prep-data.ipynb</a>.
    <li>Geocoded addresses to identify lat and long coordinates, and spatially joined addresses to Community Areas shapefile. I did this offline in QGIS.
        </ol>

In [2]:
df_dockets = pd.read_csv("../data/04-standardized/fines-by-docket.csv")
df_dockets.head()

,docket,dept,address,lat,long,community,violation_date,season,n_records,total_fine
0,19DS68300L,STRTSAN,4710 S WESTERN AVE,41.807859,-87.684797,BRIGHTON PARK,2019/11/13,2019-2020,1,150.0
1,19DS69216L,STRTSAN,1425 W MORSE AVE,42.007451,-87.666828,ROGERS PARK,2019/11/13,2019-2020,1,50.0
2,19DS70010L,STRTSAN,715 E 47TH ST,41.809338,-87.608013,GRAND BOULEVARD,2019/11/13,2019-2020,1,150.0
3,19DS72153L,STRTSAN,300 W WASHINGTON ST,41.882868,-88.210529,LOOP,2019/11/12,2019-2020,3,650.0
4,19DS72160L,STRTSAN,6929 N SHERIDAN RD,41.959813,-87.654693,UPTOWN,2019/11/14,2019-2020,1,500.0


In [3]:
len(df_dockets)

1912

# Summary by Department

In [4]:
# by issuing department
df_dockets_summary = df_dockets.groupby('dept').agg(
    total_fine=('total_fine', 'sum'),
    n_dockets=('docket','count'),
    n_records=('n_records','sum')
).reset_index()
df_dockets_summary['pct_dockets']=df_dockets_summary['n_dockets']/df_dockets_summary['n_dockets'].sum()
df_dockets_summary

,dept,total_fine,n_dockets,n_records,pct_dockets
0,BAFCONP,0.0,2,2,0.001046
1,POLICE,1700.0,25,30,0.013075
2,STRTSAN,176610.0,497,659,0.259937
3,TRANPORT,406959.0,1388,1865,0.725941


# Summarize by total fine amount

In [5]:
df_fines = df_dockets.groupby('total_fine').size().reset_index(name='n_dockets')

In [6]:
chart = alt.Chart(df_fines).mark_bar().encode(
    x=alt.X('total_fine:O', title = 'Binned Fine'),
    y=alt.Y('n_dockets:Q', title='Number of Dockets')
).properties(
    width=600,
    height=400
)

chart.display()

alt.Chart(...)

In [7]:
bins = [0, 1, 150, 151, 500, 501, 8301]
labels = ['0', '1-149', '150', '151-499', '500', '501-8300']

df_dockets['binned_fine'] = pd.cut(df_dockets['total_fine'], bins=bins, labels=labels, right=False, include_lowest=True)

df_fines_binned = df_dockets.groupby('binned_fine')['docket'].count().reset_index(name='n_dockets')
df_fines_binned

/tmp/ipykernel_1703/2177724273.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_fines_binned = df_dockets.groupby('binned_fine')['docket'].count().reset_index(name='n_dockets')


,binned_fine,n_dockets
0,0,187
1,1-149,196
2,150,705
3,151-499,67
4,500,546
5,501-8300,211


In [8]:
chart = alt.Chart(df_fines_binned).mark_bar().encode(
    x=alt.X('binned_fine:O', title = 'Binned Fine'),
    y=alt.Y('n_dockets:Q', title='Number of Dockets')
).properties(
    width=600,
    height=400
)

chart.display()

alt.Chart(...)